In [59]:
from selenium import webdriver 
import time
from tqdm.notebook import tqdm
import pandas as pd
import json, csv

import warnings
warnings.filterwarnings('ignore')

In [60]:
df = pd.read_csv('combined.csv')

In [61]:
df.head()

,dimensions,num_beds,num_baths,house_type,city,location,area_block,description,url,price,unique_id
0,"2,055 sqft",4,4,Apartment,Dhaka,Bashundhara R-A,Block D,2055 Sq Ft Cozy Flat For Rent In Bashundhara R...,https://www.bproperty.com/en/property/details-...,35000.0,5237946
1,"1,911 sqft",3,3,Apartment,Dhaka,Bashundhara R-A,Block A,Emanate Your Knack For Gardening By Renting Th...,https://www.bproperty.com/en/property/details-...,45000.0,3875341
2,"1,200 sqft",3,3,Apartment,Dhaka,Bashundhara R-A,Block K,"To Secure Your Better State Of Living, Conside...",https://www.bproperty.com/en/property/details-...,20000.0,5182996
3,"1,910 sqft",3,3,Apartment,Dhaka,Bashundhara R-A,Block A,For Rental Purpose 1910 Sq Ft Commendable Desi...,https://www.bproperty.com/en/property/details-...,45000.0,3875358
4,800 sqft,2,2,Apartment,Dhaka,Bashundhara R-A,Block G,Bright And Cozy Apartment Featuring 800 Sq Ft ...,https://www.bproperty.com/en/property/details-...,18000.0,3651253


In [62]:
completed = pd.read_csv('tertiary.csv', header=None)

In [63]:
completed.columns = ['iteration_id', 'unique_id', 'schools', 'restaurants', 'hospitals', 'parks']
completed

,iteration_id,unique_id,schools,restaurants,hospitals,parks
0,0,5237946,7,4,3,4
1,1,3875341,12,8,4,8
2,2,5182996,0,0,0,1
3,3,3875358,12,8,4,8
4,4,3651253,6,4,1,4
...,...,...,...,...,...,...
13410,4514,1667106,12,17,2,13
13411,4515,1665314,22,48,2,14
13412,4516,1664303,26,48,2,11
13413,0,1945470,23,27,3,14


In [64]:
completed_id = completed['unique_id'].values.tolist()
completed_id[:5]

[5237946, 3875341, 5182996, 3875358, 3651253]

In [65]:
df.shape

(13415, 11)

In [66]:
df = df[ ~df['unique_id'].isin(completed_id) ]

In [67]:
df.head()

,dimensions,num_beds,num_baths,house_type,city,location,area_block,description,url,price,unique_id


In [55]:
df = df.reset_index()

In [57]:
driver = webdriver.Edge(executable_path='driver/msedgedriver.exe')
tertiary_scrape = {}
bad_id = []

for i in tqdm(range(len(df))):
    try:
        url = df['url'][i]
        title = driver.get(url)

        # Click on JS Map
        driver.find_element_by_xpath('//div[@class="_4866b17e"]').click()
        time.sleep(3)

        # Click on required tabs and retrieve data
        schools = []
        restaurants = []
        hospitals = []
        parks = []

        tabs = driver.find_elements_by_xpath('//li[@class="a17d0d0d"]')

        for j in range(4):
            tabs[j].click()
            time.sleep(3)

            nearby = driver.find_elements(by='xpath', value='//div[@class="daf61ad1"]')

            if j == 0:
                for item in nearby:
                    schools.append(item.text)

            if j == 1:
                for item in nearby:
                    restaurants.append(item.text)

            if j == 2:
                for item in nearby:
                    hospitals.append(item.text)

            if j == 3:
                for item in nearby:
                    parks.append(item.text) 

        tertiary_scrape[ df['unique_id'][i] ] = {'schools': len(list(set(schools))),
                                                 'restaurants': len(list(set(restaurants))),
                                                 'hospitals': len(list(set(hospitals))),
                                                 'parks': len(list(set(parks)))}
        
        fields = [i, df['unique_id'][i], len(list(set(schools))), len(list(set(restaurants))),
                                      len(list(set(hospitals))), len(list(set(parks)))]
        
        with open(r'tertiary.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow(fields)
            f.close()
            
        if i%100 == 0 and i!=0:
            print("Resting time: 10 mins (To avoid Bot Detection)")
            print()
            driver.quit()
            for s in tqdm(range(600)):
                time.sleep(1)
            driver = webdriver.Edge(executable_path='driver/msedgedriver.exe')
            
    except Exception as e:
        print('ID: ' + str(df['unique_id'][i]))
        bad_id.append(df['unique_id'][i])
        driver.quit()
        
        print("Retrying in: 10 mins")
        for s in tqdm(range(600)):
            time.sleep(1)
        
        
        print('Starting New Instance')
        print()
        
        driver = webdriver.Edge(executable_path='driver/msedgedriver.exe')
        

driver.quit()

  0%|          | 0/2 [00:00<?, ?it/s]

In [58]:
bad_id

[]